In [8]:
import os
import cv2
import numpy as np
from scipy import ndimage
import nibabel as nib

In [9]:
def skull_strip(image_path):
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Adaptive Thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)

    # Morphological operations to close gaps
    kernel = np.ones((5,5), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 4)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations = 2)

    # Apply mask to the original image
    result = cv2.bitwise_and(image, image, mask=opening)
    return result

In [10]:
def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for subdir, dirs, files in os.walk(input_dir):
        for file in files:
            input_path = os.path.join(subdir, file)
            output_path = os.path.join(output_dir, file)

            # Process image
            processed_image = skull_strip(input_path)

            # Save image
            cv2.imwrite(output_path, processed_image)

In [11]:
# Paths
input_base = 'filtered_dataset'
output_base = 'Brain_only'

# Process each class
for class_name in ['Dementia', 'Not Dementia']:
    input_dir = os.path.join(input_base, class_name)
    output_dir = os.path.join(output_base, class_name)
    process_directory(input_dir, output_dir)

print("Processing complete.")

KeyboardInterrupt: 

In [ ]:
# # Load image
# image = cv2.imread('path_to_your_image', cv2.IMREAD_GRAYSCALE)

# # Gaussian blur
# blurred = cv2.GaussianBlur(image, (5, 5), 0)

# # Thresholding
# ret, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# # Morphological operations
# kernel = np.ones((5,5),np.uint8)
# closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2)

# # Remove small white noises
# opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations = 2)

# # Sure background area
# sure_bg = cv2.dilate(opening, kernel, iterations=3)

# # Finding sure foreground area
# dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
# ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# # Finding unknown region
# sure_fg = np.uint8(sure_fg)
# unknown = cv2.subtract(sure_bg,sure_fg)

# # Labeling regions
# ret, markers = cv2.connectedComponents(sure_fg)

# # Add one to all labels so that sure background is not 0, but 1
# markers = markers+1

# # Now, mark the region of unknown with zero
# markers[unknown==255] = 0

# # Finally apply watershed
# markers = cv2.watershed(cv2.cvtColor(image, cv2.COLOR_GRAY2RGB), markers)
# image[markers == -1] = [255,0,0]

# # Applying the mask to get the segmented brain
# mask = np.zeros_like(image)
# mask[markers == 1] = 255
# result = cv2.bitwise_and(image, image, mask=mask)

# # Show result
# cv2.imshow('Skull stripped image', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()